In [20]:
import csv
''' Depois de tratar o texto, e guardar os csv's em (letras.csv), (nutricao.csv)
    Podemos de fato começar a aplicar o método do simplex para determinar as melhores dietas'''

nomes = ['HENRIQUE','LUCAS','RAFAEL'] #nossos nomes

#nutrientes da que nossa dieta precisa satisfazer 
nutrientes_minimos = ['Caloria','Calcio','VitaminaA','Riboflavina','AcidoAscorbico'] 

dieta_prodx_por_letra = dict() #dicionario de letra para dieta e prodx
with open('letras.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dieta_prodx_por_letra[row['Letra']] = (float(row['Dieta']),float(row['ProdutoX']))

Vamos definir agora nosso problema de minimização!

Se gostaríamos de reduzir o custo, nossas variáveis serão todas $x_i$, onde $x_i$ significa quantos dólares gastamos no alimento $i$.
Como o médico nos passou quantidades mínimas para suprir de nutrientes, essas quantidades formarão nossa matriz de restrição.

Teremos algo do tipo:
$$
    x_1c_1 + x_2c_2 + ... + x_nc_n \geq c\\
    x_1p_1 + x_2p_2 + ... + x_np_n \geq p\\
    x_1v_1 + x_2v_2 + ... + x_nv_n \geq v\\
    x_1r_1 + x_2r_2 + ... + x_nr_n \geq r\\
    x_1a_1 + x_2a_2 + ... + x_na_n \geq a\\
$$

Onde $c_i$ é quanto de calórias o alimento $i$ dá por unidade de dolar gasto e $c$ é a quantidade mínima de calorias que o médico nos recomendou. (Fiz a mesma coisa para os outros nutrientes)

Como isso não está exatamente na forma padrão, teremos que multiplicar por -1 todas as linhas, de forma a tornar todo $\geq$ em $\leq$.

In [52]:
''' ideia 
M = [ [c1,c2,c3,...,cn],
      [p1,p2,p3,...,pn],
      ...
    ]
'''
alimentos = []
with open('nutricao.csv','r',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        alimentos.append(row)

In [53]:
matriz_sem_prod_x = []
for nutriente in nutrientes_minimos:
    linha_nutri = [-float(x[nutriente]) for x in alimentos]
    matriz_sem_prod_x.append(linha_nutri)


In [48]:
#com a matriz de restrição sem o produto X pronta, já estamos quase finalizados, basta descobrir quanto produto X nos dá e o B para cada nome!
from copy import deepcopy
import numpy as np
import scipy.optimize

def matriz(nome:str):
    m = deepcopy(matriz_sem_prod_x)
    for i in range(5): #são cinco nutrientes
        if i < len(nome):
            letra = nome[i].upper()
            m[i].append(-float(dieta_prodx_por_letra[letra][1]))
        else: 
            m[i].append(0.)
    return np.array(m)

def restricoes(nome):
    r = []
    for i in range(5):
        if i < len(nome):
            letra = nome[i].upper()
            r.append(-float(dieta_prodx_por_letra[letra][0]))
        else:
            r.append(0.)
    return np.array(r)



def dieta(nome:str,method:str):
    m = matriz(nome)
    r = restricoes(nome)
    c = np.ones(len(alimentos) + 1) # esse é o funcional linear que será minizado
    return scipy.optimize.linprog(c=c,A_ub=m,b_ub=r,method=method)

In [51]:
dieta('henrique','highs')

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.7160984750676888
              x: [ 8.436e-02  0.000e+00 ...  0.000e+00  6.317e-01]
            nit: 3
          lower:  residual: [ 8.436e-02  0.000e+00 ...  0.000e+00
                              6.317e-01]
                 marginals: [ 0.000e+00  6.998e-01 ...  3.448e-01
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.736e+01  1.297e+01  8.228e+01  0.000e+00
                              0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 -0.000e+00 -1.464e-02
                             -4.279e-04]
 mip_node_count: 0
 mip_dual_bound: 0.